In [1]:
from sklearn.model_selection import train_test_split
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from keras.models import Sequential
from keras.layers import Dense, LSTM
from transformers import AutoTokenizer, AutoModel
from tqdm.notebook import tqdm
from keras.utils import to_categorical

Using TensorFlow backend.


In [2]:
import torch
xlmr = torch.hub.load('pytorch/fairseq', 'xlmr.large')
xlmr.eval()

Using cache found in /home/nikamanth/.cache/torch/hub/pytorch_fairseq_master


RobertaHubInterface(
  (model): RobertaModel(
    (decoder): RobertaEncoder(
      (sentence_encoder): TransformerSentenceEncoder(
        (embed_tokens): Embedding(250002, 1024, padding_idx=1)
        (embed_positions): LearnedPositionalEmbedding(514, 1024, padding_idx=1)
        (layers): ModuleList(
          (0): TransformerSentenceEncoderLayer(
            (self_attn): MultiheadAttention(
              (k_proj): Linear(in_features=1024, out_features=1024, bias=True)
              (v_proj): Linear(in_features=1024, out_features=1024, bias=True)
              (q_proj): Linear(in_features=1024, out_features=1024, bias=True)
              (out_proj): Linear(in_features=1024, out_features=1024, bias=True)
            )
            (self_attn_layer_norm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
            (fc1): Linear(in_features=1024, out_features=4096, bias=True)
            (fc2): Linear(in_features=4096, out_features=1024, bias=True)
            (final_layer_norm): 

In [3]:
train_df = pd.read_csv('../NADI-2020_release_1.0/NADI_release/train_labeled.tsv',sep='\t')
dev_df = pd.read_csv('../NADI-2020_release_1.0/NADI_release/dev_labeled.tsv',sep='\t')

X_train_original,y_train_original = train_df["#2 tweet_content"],train_df["#3 country_label"]
X_dev_original,y_dev_original = dev_df["#2 tweet_content"],dev_df["#3 country_label"]

In [4]:
labels = {}
uni = y_train_original.unique()
for i in range(len(uni)):
    labels[uni[i]] = i

y_train_index = [labels[i] for i in y_train_original]

y_dev_index = [labels[i] for i in y_dev_original]

In [5]:
X_train = []
for i in tqdm(X_train_original):
    ar_tokens = xlmr.encode(i)
    last_layer_features = xlmr.extract_features(ar_tokens).detach().numpy()
    X_train.append(last_layer_features[0])
np.array(X_train)

X_dev = []
for i in tqdm(X_dev_original):
    ar_tokens = xlmr.encode(i)
    last_layer_features = xlmr.extract_features(ar_tokens).detach().numpy()
    X_train.append(last_layer_features[0])
np.array(X_dev)

KeyboardInterrupt: 

In [ ]:
X_train = np.array(X_train)
X_dev = np.array(X_dev)

In [ ]:
y_train = to_categorical(y_train_index, num_classes=21)
y_dev = to_categorical(y_dev_index, num_classes=21)

In [ ]:
model = Sequential()
model.add(LSTM(100,dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(21, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [98]:
history = model.fit(X_train,y_train,epochs=200)

Epoch 1/500
4/4 [==============================] - 0s 121ms/step - loss: 2.5220 - accuracy: 0.2500
Epoch 2/500
4/4 [==============================] - 0s 15ms/step - loss: 2.4752 - accuracy: 0.2500
Epoch 3/500
4/4 [==============================] - 0s 17ms/step - loss: 1.9085 - accuracy: 0.5000
Epoch 4/500
4/4 [==============================] - 0s 15ms/step - loss: 1.9443 - accuracy: 0.5000
Epoch 5/500
4/4 [==============================] - 0s 16ms/step - loss: 1.8264 - accuracy: 0.5000
Epoch 6/500
4/4 [==============================] - 0s 15ms/step - loss: 1.4801 - accuracy: 0.5000
Epoch 7/500
4/4 [==============================] - 0s 15ms/step - loss: 1.7006 - accuracy: 0.5000
Epoch 8/500
4/4 [==============================] - 0s 14ms/step - loss: 1.4531 - accuracy: 0.5000
Epoch 9/500
4/4 [==============================] - 0s 15ms/step - loss: 1.4587 - accuracy: 0.5000
Epoch 10/500
4/4 [==============================] - 0s 14ms/step - loss: 1.3706 - accuracy: 0.5000
Epoch 11/500
4/4 [

Epoch 84/500
4/4 [==============================] - 0s 15ms/step - loss: 1.0571 - accuracy: 0.5000
Epoch 85/500
4/4 [==============================] - 0s 15ms/step - loss: 1.0433 - accuracy: 0.5000
Epoch 86/500
4/4 [==============================] - 0s 15ms/step - loss: 0.9676 - accuracy: 0.5000
Epoch 87/500
4/4 [==============================] - 0s 13ms/step - loss: 0.9517 - accuracy: 0.5000
Epoch 88/500
4/4 [==============================] - 0s 15ms/step - loss: 1.0418 - accuracy: 0.5000
Epoch 89/500
4/4 [==============================] - 0s 13ms/step - loss: 1.1368 - accuracy: 0.5000
Epoch 90/500
4/4 [==============================] - 0s 12ms/step - loss: 1.1217 - accuracy: 0.5000
Epoch 91/500
4/4 [==============================] - 0s 15ms/step - loss: 1.2260 - accuracy: 0.5000
Epoch 92/500
4/4 [==============================] - 0s 15ms/step - loss: 1.0250 - accuracy: 0.5000
Epoch 93/500
4/4 [==============================] - 0s 15ms/step - loss: 1.1023 - accuracy: 0.5000
Epoch 94/5

KeyboardInterrupt: 

In [8]:
t1 = time.time()
ar_tokens = xlmr.encode(i)
last_layer_features = xlmr.extract_features(ar_tokens).detach().numpy()
X_train.append(last_layer_features[0])
t2 = time.time()
t2-t1

0.3618049621582031